# Data-Driven Decision Making in SQL

Here you can access every table used in the course. To access each table, you will need to specify the `movies` schema in your queries (e.g., `movies.movies` for the `movies` table, and `movies.customers` for the `customers` table).

In [5]:
-- check the tables in the table_schema='movies'
SELECT table_name, table_schema, table_type
FROM information_schema.tables
where table_schema = 'movies'
ORDER BY table_name ASC;

,table_name,table_schema,table_type
0,actors,movies,BASE TABLE
1,actsin,movies,BASE TABLE
2,customers,movies,BASE TABLE
3,movies,movies,BASE TABLE
4,renting,movies,BASE TABLE


## Screen the tables and the columns

In [1]:
-- explore the tables
SELECT *
FROM movies.actors
LIMIT 5

,actor_id,name,year_of_birth,nationality,gender
0,1,Abbie Cornish,1982,Australia,female
1,2,Adam Sandler,1966,USA,male
2,3,Al Pacino,1940,USA,male
3,4,Amy Adams,1974,USA,female
4,5,Andrea Riseborough,1981,British,female


In [3]:
select * 
from movies.movies
limit 5

,movie_id,title,genre,runtime,year_of_release,renting_price
0,1,One Night at McCool's,Comedy,93,2001,2.09
1,2,Swordfish,Drama,99,2001,2.19
2,3,What Women Want,Comedy,127,2001,2.59
3,4,Training Day,Drama,122,2001,1.79
4,5,The Fellowship of the Ring,Science Fiction & Fantasy,178,2001,2.59


In [2]:
select *
from movies.renting 
limit 5

,renting_id,customer_id,movie_id,rating,date_renting
0,1,41,8,NaN,2018-10-09 00:00:00+00:00
1,2,10,29,10.0,2017-03-01 00:00:00+00:00
2,3,108,45,4.0,2018-06-08 00:00:00+00:00
3,4,39,66,8.0,2018-10-22 00:00:00+00:00
4,5,104,15,7.0,2019-03-18 00:00:00+00:00


In [2]:
select *
from movies.customers
limit 5

,customer_id,name,country,gender,date_of_birth,date_account_start
0,2,Wolfgang Ackermann,Austria,male,1971-11-17 00:00:00+00:00,2018-10-15 00:00:00+00:00
1,3,Daniela Herzog,Austria,female,1974-08-07 00:00:00+00:00,2019-02-14 00:00:00+00:00
2,4,Julia Jung,Austria,female,1991-01-04 00:00:00+00:00,2017-11-22 00:00:00+00:00
3,5,Juliane Kirsch,Austria,female,1977-03-01 00:00:00+00:00,2018-12-16 00:00:00+00:00
4,6,Rowanne Couperus,Belgium,female,1994-04-05 00:00:00+00:00,2018-08-26 00:00:00+00:00


In [1]:
select *
from movies.actsin
limit 5

,actsin_id,movie_id,actor_id
0,1,37,1
1,2,56,2
2,3,10,3
3,4,14,3
4,5,29,3


## Explore the renting table to see the number of renting records, number of distinct customer, and number of ratings

In [3]:
--renting table is the main table which store the renting data for the store
select 
count(*) as no_renting
, count(distinct customer_id) as unique_customer
, count(rating) as no_rating
from movies.renting

,no_renting,unique_customer,no_rating
0,578,116,328


## Average rating per movie
For each movie the average rating, the number of ratings and the number of views has to be reported
<br> Join movies table for the movie title

In [5]:
select
m.title
, avg(rating) as avg_rating
, count(rating) as no_rating
, count(*) as no_renting
from movies.renting as r
left join movies.movies as m
ON r.movie_id = m.movie_id
group by m.title
having  avg(rating) is not null   -- after grouping by title, remove the movies which do not have any avg_rating
order by avg_rating DESC          --check the highest rating movie

,title,avg_rating,no_rating,no_renting
0,Astro Boy,10.000000,2,5
1,The Fellowship of the Ring,9.750000,4,8
2,No Country for Old Men,9.600000,5,10
3,Django Unchained,9.333333,6,11
4,What Women Want,9.000000,2,11
...,...,...,...,...
65,The Recruit,6.500000,6,11
66,The Company You Keep,6.333333,3,7
67,The Invasion,6.000000,1,4
68,Brighton Rock,6.000000,2,5


## Average rating per customer
We will report these summary statistics only for customers with more than 7 movie rentals and order them in ascending order by the average rating.

In [26]:
select 
c.name
, avg(r.rating) as avg_rating
, count(r.rating) as no_rating
, count(*) as no_renting
from movies.renting as r
left join movies.customers as c
ON r.customer_id = c.customer_id
group by c.name
having count(*) >7
order by no_renting DESC

,name,avg_rating,no_rating,no_renting
0,Lucy Centeno Barrios,7.000000,7,15
1,Avelaine Corbeil,7.333333,6,14
2,Havasy Kristof,7.625000,8,13
3,Sidney Généreux,6.714286,7,11
4,Canela Gaona Lozano,8.000000,6,11
5,Honorata Nowak,7.571429,7,11
6,Saúl Tafoya Meraz,8.000000,6,10
7,Marcel Sicard,8.400000,5,10
8,Fiacre Montminy,7.200000,5,10
9,Filippa Toscani,8.000000,7,10


### Financial successes & User engagement
Report key performance indicators (KPIs) for the performance of the company in 2018.
<br>Financial successes and user engagement. 
<br>Important KPIs are, the profit coming from movie rentals, the number of movie rentals and the number of active customers.

In [58]:
--2018 where the profit comes from 
select 
m.title
, count(*) as no_renting
, sum(renting_price) as revenue
from movies.renting as r
left join movies.movies as m
on r.movie_id = m.movie_id
where r.date_renting between '2018-01-01' AND '2018-12-31'
group by m.title
order by revenue DESC

,title,no_renting,revenue
0,Bridget Jones - The Edge of Reason,9,26.01
1,Fair Game,7,20.23
2,Two for the Money,6,16.74
3,Harry Potter and the Deathly Hallows – Part 1,6,16.74
4,Training Day,9,16.11
...,...,...,...
66,The Human Stain,2,3.98
67,Happy Feet,2,3.58
68,Something's Gotta Give,2,3.38
69,The Invasion,1,2.09


In [24]:
--select the top1 in 2018
with table2018 as(
select 
extract (year from r.date_renting) as year
, m.title
, count(*) as no_renting
, sum(renting_price) as revenue
from movies.renting as r
left join movies.movies as m
on r.movie_id = m.movie_id
where r.date_renting between '2018-01-01' AND '2018-12-31'
group by m.title, year)


select * from table2018
where revenue = (select Max(revenue) from table2018)

,year,title,no_renting,revenue
0,2018,Bridget Jones - The Edge of Reason,9,26.01


In [26]:
--select the top1 in 2019 using other method
select 
title
, count(*) as no_renting
, sum(renting_price) as revenue
from movies.renting as r
left join movies.movies as m
on r.movie_id = m.movie_id
group by title
having sum(renting_price)=
    (select
    max(table2019.revenue)
    from
        (select 
            m.title
            , count(*) as no_renting
            , sum(renting_price) as revenue
            from movies.renting as r
            left join movies.movies as m
            on r.movie_id = m.movie_id
            where r.date_renting between '2019-01-01' AND '2019-12-31'
            group by m.title) as table2019)

,title,no_renting,revenue
0,Astro Boy,5,14.45


In [35]:
-- to see the yearly performance
select 
extract(year from date_renting) as year
, count(*) as no_renting
, sum(renting_price) as revenue
from movies.renting as r
left join movies.movies as m
on r.movie_id = m.movie_id
group by year
order by year

,year,no_renting,revenue
0,2017,121,263.19
1,2018,298,658.02
2,2019,159,354.51


## Explore the movies table

In [32]:
--see overall max, min avg renting price
select 
max(renting_price)
, min(renting_price)
, avg(renting_price)
from movies.movies

,max,min,avg
0,2.99,1.49,2.209718


In [35]:
select 
title
, renting_price
from movies.movies
where renting_price = (select max(renting_price) from movies.movies)

,title,renting_price
0,11'09''01 September 11,2.99
1,Morning Glory,2.99


## Favorite actors
Who plays most often in movies watched by customers?
- Actor being watched most often
- best avg rating when being watched

In [6]:
-- Actors being watched most often
select 
a.gender
, a.name
, count(*) as number_views
from movies.renting as r
left join movies.actsin as ai
on r.movie_id = ai.movie_id
left join movies.actors as a
on ai.actor_id = a.actor_id
group by a.gender, a.name
order by count(*) DESC

,gender,name,count
0,male,Daniel Radcliffe,71
1,female,Emma Watson,71
2,male,Rupert Grint,71
3,male,Jamie Foxx,42
4,female,Helen Hunt,37
...,...,...,...
140,male,Jeremy Northam,4
141,female,Katie Holmes,4
142,male,Liam Neeson,4
143,male,Scott Cohen,2


In [2]:
--Actors being rat
select 
a.gender
, a.name
, avg(rating) as avg_rating
, count(*) as number_views
from movies.renting as r
left join movies.actsin as ai
on r.movie_id = ai.movie_id
left join movies.actors as a
on ai.actor_id = a.actor_id
group by a.gender, a.name
having avg(rating) is not null
order by avg_rating DESC

,gender,name,avg_rating,count
0,female,Kristen Bell,10.000000,5
1,male,Freddie Highmore,10.000000,5
2,male,Tommy Lee Jones,9.600000,10
3,male,Josh Brolin,9.600000,10
4,male,Javier Bardem,9.600000,10
...,...,...,...,...
138,male,Jeremy Northam,6.000000,4
139,female,Andrea Riseborough,6.000000,5
140,male,Daniel Craig,6.000000,4
141,female,Katie Holmes,5.666667,4


## Identify favorite movies for a group of customers

In [36]:
select 
max(c.date_of_birth)
, min(c.date_of_birth)
from movies.renting as r
left join movies.customers as c
ON r.customer_id = c.customer_id

,max,min
0,1999-07-21 00:00:00+00:00,1970-01-10 00:00:00+00:00


In [14]:
--Select only those records of customers born in the 70s.
select
m.title
, count(*) as number_renting
, avg(r.rating) as average_rating
from movies.renting as r
left join movies.movies as m
on r.movie_id = m.movie_id
left join movies.customers as c
on r.customer_id = c.customer_id
where c.date_of_birth between '1970-01-01' AND '1979-12-31'
group by m.title
having count(*) >1
order by average_rating DESC


,title,number_renting,average_rating
0,Showtime,5,NaN
1,Harry Potter and the Deathly Hallows – Part 2,2,NaN
2,Waking Up in Reno,2,NaN
3,Ray,2,NaN
4,Django Unchained,4,10.000000
5,The Fighter,4,10.000000
6,One Night at McCool's,2,10.000000
7,No Country for Old Men,3,10.000000
8,The Fellowship of the Ring,2,10.000000
9,I'm Not There,2,10.000000
